In [ ]:
%%shell
pip install pymanopt
pip install git+https://github.com/geoopt/geoopt.git
if [ -d NeuralCollapse ]
then
    rm -rf NeuralCollapse
fi
git clone https://github.com/10258392511/NeuralCollapse.git
cp NeuralCollapse/problem.py /usr/local/lib/python3.7/dist-packages/pymanopt/core/problem.py
cp -r NeuralCollapse/* .

%load_ext autoreload 
%autoreload 2

In [33]:
import numpy as np
import torch
import geoopt
import pickle

from run_cifar100.demos import demo_manifold_tensor, demo_manifold_param
demo_manifold_tensor()
# demo_manifold_param()

W: Tensor on Sphere manifold containing:
tensor([[-0.9497,  0.3132],
        [ 0.3278, -0.9448],
        [ 0.8772,  0.4801]], requires_grad=True)
norm: tensor([1., 1., 1.], grad_fn=<CopyBackwards>)
W_next: tensor([[-0.9739,  0.2268],
        [ 0.2893, -0.9572],
        [ 0.9104,  0.4137]])
norm: tensor([1.0000, 1.0000, 1.0000])


In [37]:
## get datasets
from run_cifar100.datasets import select_k_cls
# slecet k class,dim is not fixed in this sample 
BATCH_SIZE = 128
train_loader, test_loader = select_k_cls(num_cls=5, batch_size=BATCH_SIZE, if_plot_batch=False)



Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [40]:
# test of dataset 
from torchvision.datasets import CIFAR100
num_cls = 10
tmp_train_dataset = CIFAR100("./data", transform=None, download=True)
classes = np.random.choice(len(tmp_train_dataset.classes), num_cls, replace=False)
print("classes: {}".format(classes))


Files already downloaded and verified
classes: [59 36 48 23  8 29  6 42 14 61]


In [17]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device: {}".format(DEVICE))

from run_cifar100.models import ResNetAdapt # use resnets for function
M, K = 2, 5 # M means manifold, K means class_numbers
model = ResNetAdapt(M, K, True, True)

opt = geoopt.optim.RiemannianAdam([param for param in model.parameters() if param.requires_grad])
step_size = 10
gamma = 0.8
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=step_size, gamma=gamma)

device: cpu


In [18]:
from train_K_gt_d import train_cli

In [22]:
K = 4
feature_man = True
weight_man = True
weight_alpha = 0.1

args = {"M": 2, 
    "K": K, 
    "epoch": 10, 
    "batch_size": 128, 
    "lr": 2e-2,
    "scheduler": False,
    "feature_man": feature_man,
    "W_man": weight_man,
    'weight_alpha': weight_alpha}
        
train_cli(**args)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html